<a href="https://colab.research.google.com/github/Samin-Sadaf7/ObjectDetectiongwithYolo/blob/main/yolov8_object_detection_with_roboflow_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# YOLOv8 Object Detection on a Dataset

---

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset)
[![YouTube](https://badges.aleen42.com/src/youtube.svg)](https://youtu.be/wuZtUMEiKWY)
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/ultralytics/ultralytics)

Ultralytics YOLOv8 is the latest version of the YOLO (You Only Look Once) object detection and image segmentation model developed by Ultralytics. The YOLOv8 model is designed to be fast, accurate, and easy to use, making it an excellent choice for a wide range of object detection and image segmentation tasks. It can be trained on large datasets and is capable of running on a variety of hardware platforms, from CPUs to GPUs.

## ⚠️ Disclaimer

YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **03.01.2024** with version **YOLOv8.0.196**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

## Accompanying Blog Post

We recommend that you follow along in this notebook while reading the blog post on how to train YOLOv8 Object Detection, concurrently.

## Pro Tip: Use GPU Acceleration

If you are running this notebook in Google Colab, navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`. This will ensure your notebook uses a GPU, which will significantly speed up model training times.

## Steps in this Tutorial

In this tutorial, we are going to cover:

- Before you start
- Install YOLOv8
- CLI Basics
- Inference with Pre-trained COCO Model
- Roboflow Universe
- Preparing a custom dataset
- Custom Training
- Validate Custom Model
- Inference with Custom Model

**Let's begin!**

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

Wed Aug 14 17:35:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **03.01.2024** with version **YOLOv8.0.196**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.6/107.7 GB disk)


In [ ]:
# Git clone method (for development)

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -e .

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Inference with Pre-trained COCO Model

### 💻 CLI

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the latest YOLOv8 release, and saving results to `runs/predict`.

## Training on Traffic Sign Detection Dataset

In [ ]:
%cd /content/

/content


In [ ]:
import os
import shutil

# Define paths
xml_folder = 'label'
img_folder = 'images'

# Create folders if they don't exist
os.makedirs(xml_folder, exist_ok=True)
os.makedirs(img_folder, exist_ok=True)

# Get a list of all XML files in the current directory
xml_files = [file for file in os.listdir() if file.endswith('.xml')]

# Get a list of all PNG files in the current directory
png_files = [file for file in os.listdir() if file.endswith('.png')]

# Move XML files to the 'label' folder
for xml_file in xml_files:
    shutil.move(xml_file, os.path.join(xml_folder, xml_file))

# Move PNG files to the 'images' folder
for png_file in png_files:
    shutil.move(png_file, os.path.join(img_folder, png_file))

In [ ]:
import yaml

# Define the data
data = {
    'nc': 15,
    'train': '/content/drive/MyDrive/Dataset- -Conference/train/',
    'val': '/content/drive/MyDrive/Dataset- -Conference/val/',
    'names': [
        'SpeedLimit60kmh',
        'School',
        'PEDESTRIANCROSSING',
        'SharpBendToTheLeft',
        'UTurn',
        'speedlimit80kmh',
        'NoOvertaking',
        'SharpBendToTheRight',
        'SideRoadLeft',
        'SideRoadRight',
        'speedlimit40kmh',
        'NarrowBridge',
        'RoadHump',
        'nouseofhorn',
        'Crossroads'
    ]
}

# Write the data to conference.yml
with open('conference.yml', 'w') as yaml_file:
    yaml.dump(data, yaml_file)


In [ ]:
import xml.etree.ElementTree as ET
import os

# Class names
class_names =[
        'SpeedLimit60kmh',
        'School',
        'PEDESTRIANCROSSING',
        'SharpBendToTheLeft',
        'UTurn',
        'speedlimit80kmh',
        'NoOvertaking',
        'SharpBendToTheRight',
        'SideRoadLeft',
        'SideRoadRight',
        'speedlimit40kmh',
        'NarrowBridge',
        'RoadHump',
        'nouseofhorn',
        'Crossroads'
    ]

# Create class dictionary
class_dict = {class_name: i for i, class_name in enumerate(class_names)}

def xml_to_yolo(xml_file, class_dict):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_size = root.find('size')
    width = float(image_size.find('width').text)
    height = float(image_size.find('height').text)

    yolo_annotations = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text.strip().replace(' ', '').replace('/', '')  # Remove spaces and '/'
        class_id = class_dict.get(class_name)
        if class_id is None:
            print(f"Warning: Class '{class_name}' not found in class dictionary.")
            continue

        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)

        # Calculate normalized coordinates
        x_center = (xmin + xmax) / (2 * width)
        y_center = (ymin + ymax) / (2 * height)
        bbox_width = (xmax - xmin) / width
        bbox_height = (ymax - ymin) / height

        yolo_annotations.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

    return yolo_annotations


def convert_annotations(xml_dir, output_dir, class_dict):
    os.makedirs(output_dir, exist_ok=True)

    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            img_id = os.path.splitext(xml_file)[0]
            yolo_annotations = xml_to_yolo(os.path.join(xml_dir, xml_file), class_dict)
            with open(os.path.join(output_dir, f"{img_id}.txt"), 'w') as f:
                f.write('\n'.join(yolo_annotations))

# Paths
xml_dir = '/content/drive/MyDrive/Dataset- -Conference/train/labels1'
output_dir = '/content/drive/MyDrive/Dataset- -Conference/train/labels'

# Convert annotations
convert_annotations(xml_dir, output_dir, class_dict)


In [ ]:
# train label Paths
xml_dir = '/content/drive/MyDrive/Dataset- -Conference/train/labels1'
output_dir = '/content/drive/MyDrive/Dataset- -Conference/train/labels'

# Convert annotations
convert_annotations(xml_dir, output_dir, class_dict)

In [ ]:
# valid test Paths
xml_dir = '/content/drive/MyDrive/Dataset- -Conference/val/labels1'
output_dir = '/content/drive/MyDrive/Dataset- -Conference/val/labels'

# Convert annotations
convert_annotations(xml_dir, output_dir, class_dict)

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO()  # build a new model from scratch

100%|██████████| 6.23M/6.23M [00:00<00:00, 21.4MB/s]


In [ ]:
# Train the model
results = model.train(data='/content/drive/MyDrive/BD_Road_Sign_Dataset/data.yaml', epochs=20)

100%|██████████| 6.23M/6.23M [00:00<00:00, 332MB/s]
New https://pypi.org/project/ultralytics/8.2.77 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/BD_Road_Sign_Dataset/data.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stri

In [ ]:
# Define paths
import shutil
source_folder = '/content/runs'
destination_folder = '/content/drive/MyDrive/BD_Road_Sign_Dataset/yolov8'  # Path to the destination folder
# Copy the folder and its contents to the destination
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/BD_Road_Sign_Dataset/yolov8'

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:

# Now you can run your YOLO command
from ultralytics import YOLO
!yolo task=detect mode=val model='/content/runs/detect/train/weights/best.pt' data=/content/drive/MyDrive/BD_Road_Sign_Dataset/data.yaml

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3011108 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/BD_Road_Sign_Dataset/valid/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100% 46/46 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 3/3 [00:03<00:00,  1.09s/it]
                   all         46         49      0.873      0.894      0.941      0.879
              Bus Stop         46          1      0.862          1      0.995      0.995
      Crossroads Ahead         46          2      0.985          1      0.995      0.995
               Go Left         46          2      0.915          1      0.995      0.947
               Go Slow         46          1      0.862          1      0.995      0.895
        Hospital Ahead         46          2      0.967        0.5      0.523      0.419
             Left Turn        

In [ ]:
# Define paths
import shutil
source_folder = '/content/runs/'
destination_folder = '/content/drive/MyDrive/BD_Road_Sign_Dataset/yolov8/final'  # Path to the destination folder
# Copy the folder and its contents to the destination
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/BD_Road_Sign_Dataset/yolov8/final'

In [ ]:
# Run on test set for prediction
!yolo task=detect mode=predict model='/content/runs/detect/train/weights/best.pt' conf=0.25 source='/content/drive/MyDrive/BD_Road_Sign_Dataset/test/images'

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3011108 parameters, 0 gradients, 8.1 GFLOPs

image 1/48 /content/drive/MyDrive/BD_Road_Sign_Dataset/test/images/IMG_0026_PNG.rf.7c7f008ae721f06eb25688fb54beaddb.jpg: 640x640 1 Speed Limit 60kmh, 7.7ms
image 2/48 /content/drive/MyDrive/BD_Road_Sign_Dataset/test/images/IMG_0055_PNG.rf.8e2037142b302388508bada09715736b.jpg: 640x640 1 Road On Left, 7.5ms
image 3/48 /content/drive/MyDrive/BD_Road_Sign_Dataset/test/images/IMG_0061_PNG.rf.e1e5e8e560b21d3166faa6e9ba5196ed.jpg: 640x640 1 Right Turn, 7.5ms
image 4/48 /content/drive/MyDrive/BD_Road_Sign_Dataset/test/images/Screenshot-2023-08-23-201056_jpg.rf.f338087bb3ced2ae099a8a0acf6d5252.jpg: 640x640 1 Go Slow, 7.5ms
image 5/48 /content/drive/MyDrive/BD_Road_Sign_Dataset/test/images/Screenshot-2023-08-23-203139_jpg.rf.f63f5cdedc032633e6ef3f9fb10be401.jpg: 640x640 1 Filling Station Ahead, 7.5ms
image 6/48 /content/drive/MyDr

In [ ]:
# Define paths
import shutil
source_folder = '/content/runs'
destination_folder = '/content/drive/MyDrive/BD_Road_Sign_Dataset/yolov8/test_final'  # Path to the destination folder
# Copy the folder and its contents to the destination
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/BD_Road_Sign_Dataset/yolov8/test_final'